## Fleet and workforce planning with Linear Programming
#### A preview of Linear Programming's capabilities to solve some business problems
###### Author: Kevin Esh

**Problem:** finding the optimal (less costly) way of transporting product boxes from warehouses to stores employing different vans and workers while fulfilling the constraints of demand, stock, vans and workers assignment.

**Context Variables**:
- "P" products = 5
- "W" warehouses = 3
- "S" stores = 3
- "V" vans = 4
- "E" employees = 8
- "J" conflicting pairs of employees = 23

**Assumptions**:
- We know the demand in boxes of each of our products, by store;
- We know how many boxes of our products are available in each of our warehouses;
- All the boxes have the same size (to simplify the problem);
- We know how many boxes fit in each of the vehicles;
- The cost of sending a box varies depending on the combination of product-warehouse-store-van;
- The use of each van has a fixed cost;
- The vans can't go from a warehouse to a store without transporting at least 1 box;
- The vans can't do more than 5 trips. We can assume that they all occur on the same day;
- There's a limit to how many boxes can be transported by each van;
- None of the vans can repeat the same trip;
- We need 2 employees assigned per van to be able to make a trip;
- If we assign an employee we have to pay him for this task 🙂;
- Each employee can only be assigned to a single van;
- It is desirable to avoid assigning conflicting employees to the same vehicle.

### Variables, Objective and Constraints

- $c_{pwsv}$: Unit cost of sending a box of product $p$ from warehouse $w$ to store $s$ using vehicle $v$, where $\space $   $c_{pwsv} \in (0,+\infty)$;

- $x_{pwsv}$: Number of boxes of product $p$ sent from warehouse $w$ to store $s$ using van $v$, where $\space  $ $x_{pwsv} \in [0,+\infty)$;

- $T_v$: Boolean variable created to verifiy that the vehicle $v$ is used, where $\space $ $T_v \in \{0,1\}$;

- $F_v$: Integer fixed cost for using the vehicle $v$, where $\space $ $F_v \in (0,+\infty)$;

- $A_{ve}$: Boolean variable created to verify if an employee $e$ is assigned to a van $v$, where $\space $ $A_{ve} \in \{0,1\}$;

- $Z_{wsv}$: Boolean auxiliary variable used to check if a trip from warehouse $w$ to store $s$ was made using van $v$ , where $\space $ $Z_{wsv} \in \{0,1\}$;

- $H_{vj}$: Boolean auxiliary variable to check if just one employee of a conflicting pair $j$ is assigned to a van $v$, where $\space $ $H_{vj} \in \{0,1\}$

- $G_{vj}$: Boolean variable to check if the conflicting pair $j$ is assigned to the same van $v$, where $\space $ $G_{vj} \in \{0,1\}$

$Min \sum\limits_{\substack{p\in P \\ w \in W \\ s \in S\\ v \in V}}c_{pwsv}x_{pwsv}+ \sum\limits_{\substack{v\in V}} T_{v}F_{v}+ \sum\limits_{\substack{v\in V\\ e \in E}} 1500A_{ve}+ \sum\limits_{\substack{v\in V\\ j \in J}}500G_{vj}$ $\ \$ (1)

$\ \ subject \ \ to$
 
- $\sum\limits_{\substack{w \in W \\ v \in V}} x_{pwsv} = demand_{ps}$ $\ \ \forall \ \ p \in P, \ \ s \in S$ $\ \$ (2)


- $\sum\limits_{\substack{s \in S \\ v \in V}}x_{pwsv} \le stock_{pw}$ $\ \ \forall \ \ p \in P, \ \ w \in W$ $\ \$ (3)


- $\sum\limits_{\substack{p\in P}} x_{pwsv} \le capacity_{v}Z_{wsv}$  $\ \ \forall \ \ w \in W, \ \ s \in S, \ \ v \in V$ $\ \$ (4)


- $\sum\limits_{\substack{w \in W \\ s \in S}}Z_{wsv} \le 5T_v$  $\ \ \forall \ \ v \in V$ $\ \$ (5)


- $\sum\limits_{\substack{e \in E}}A_{ve} = 2T_v$  $\ \ \forall \ \ v \in V$ $\ \$ (6)


- $\sum\limits_{\substack{v \in V}}A_{ve} \le 1$  $\ \ \forall \ \ e \in E$ $\ \$ (7)


- $\sum\limits_{\substack{v \in V}}A_{ve_1}+A_{ve_2}-H_{vj}-2G_{vj} = 0$ $\ \ \forall \ \ e \in E, \ \ j \in J \ \ $ 
with $ J =\{(e_a,e_b) \in E :(a,b) \ \ pairs \ \ with \ \ conflicts\}$ $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \ \ \ \ \ \ \ \ \ \ \ \ \ \  \  \ \ and \ \ j = \{e_a \ne e_b : (e_a,e_b) \in J\} \\ (8)$


-  $x_{pwsv} \in [0,+\infty)$ $\  \  T_{v},\ \ Z_{wsv},\ \ A_{ve},\ \ H_{vj},\ \ G_{vj} \in \{0,1\}$ $\ \$ (9)


with $p=1,...,P, \space\space w=1,...,W,\space\space s=1,...,S \space\space v=1,...,V \space\space e=1,...,E \space\space j=1,...,J$

#### Imports

In [22]:
import numpy as np
import pandas as pd
from itertools import product
from ortools.sat.python import cp_model

#### Full solution

In [176]:
#1) Specify a seed to be able to replicate the experiment
np.random.seed(2)

#2) Declare the context variables
number_warehouses = 10
number_stores = 4
number_products = 2
number_vehicles = 4
number_employees = 8
trip_limit = 10

#3) Set some thresholds for the simulated data
min_stock, max_stock = 40, 50
min_demand, max_demand = 50, 100
min_capacity, max_capacity = 15, 20


min_cost, max_cost = 1, 6
min_van_fixed_cost, max_van_fixed_cost = 100, 200
fixed_salary = 150
conflict_penalty = 50

#4) GENERATE DATA
 # 1 Cost matrix per product (of size number_warehouses x number_stores x number_vehicles)
 # 1 Stock vector per product (of size number_warehouses)
 # 1 Demand vector per product (of size number_stores)
 # 1 Capacity list per van (i.e. how many boxes each can transport)
 # 1 List of pairs of conflicting employees

capacities = [15, 20, 20, 15]
conflicted_employees = [
    (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7),
    (1,2),(1,3),(1,4),(1,5),(1,6),(1,7),
    (2,3),(2,4),(2,5),(2,6),(2,7),
    (3,4),(3,5),(3,6),(3,7),
    (4,5)
    ]


# For each product we'll generate random costs, stocks and demands, given by the intervals [low,high]
costs_shipping = np.random.randint(min_cost, max_cost, size=(number_warehouses, number_stores))
stocks = np.random.randint(min_stock, max_stock, size=(number_products, number_warehouses))
demands = np.random.randint(min_demand, max_demand, size=(number_products, number_stores))
# We create a vector of fixed costs for using each of the vans
van_fees = np.random.randint(min_van_fixed_cost, max_van_fixed_cost, size=number_vehicles)


In [177]:
fees_van

array([197, 118, 184, 190])

In [178]:
# 5) CALL AN INSTANCE OF A SOLVER FOR MIXED INTEGER PROGRAMMING PROBLEMS
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# 6) CREATE THE VARIABLES

# Variable X reflects the number of boxes shipped from a warehouse to a store using a specific van
box = {}
for p in range(number_products):
    for w in range(number_warehouses):
        for s in range(number_stores):
            for v in range(number_vehicles):
                # Write the variables following the previous mathematical notation x[Product,Warehouse,Store, Van]
                # Here we also specify that each quantity must be positive (from 0 to infinity)
                box[p, w, s, v] = model.NewIntVar(0, max(capacities), f"x_{p}_{w}_{s}_{v}")

# Variable reflects the use of a van 
is_vehicle_used = {}
for v in range(number_vehicles):
    is_vehicle_used[v] = model.NewBoolVar(f"is_used_{v}")

# Variable signals if employee e is assigned to van v
is_employee_assigned = {}
for v in range(number_vehicles):
    for e in range(number_employees):
        is_employee_assigned[v, e] = model.NewBoolVar(f"is_assigned_{v}_{e}")

# Thi is an auxiliary variable used to count the number of trips a van makes
trip = {}
for v in range(number_vehicles):
    for w in range(number_warehouses):
        for s in range(number_stores):
            trip[w, s, v] = model.NewBoolVar(f"trip_{w}_{s}_{v}")

is_vehicle_conflicted = {}
for v in range(number_vehicles):
    for _, (e1, e2) in enumerate(conflicted_employees):
        is_vehicle_conflicted[v, _] = model.NewBoolVar(f"is_conflicted_{v}_{e1}_{e2}")


In [179]:
# 7) DEFINE THE CONSTRAINTS

# The sum of the columns (stock received at each store) must equal the store's demand
for (p, s) in product(range(number_products), range(number_stores)):
    pairs = product(range(number_warehouses), range(number_vehicles))
    boxes_shipped_to_store = sum(box[p, _w, s, _v] for (_w, _v) in pairs)
    model.Add(boxes_shipped_to_store == demands[p, s]) # product_demand_in_store

# The sum of stock of product p received at each store must be lower or equal to the available stock of p at each warehouse w
for (p, w) in product(range(number_products), range(number_warehouses)):
    pairs = product(range(number_stores), range(number_vehicles))
    boxes_shipped_from_warehouse = sum(box[p, w, _s, _v] for (_s, _v) in pairs)
    model.Add(boxes_shipped_from_warehouse <= stocks[p, w])  # product stock in warehouse

# None of the vans can make more than 5 trips
# a) Register the possibility of making a trip from warehouse w to store s using van v
for v in range(number_vehicles):
    for s in range(number_stores):
        for w in range(number_warehouses):
            boxes_shipped_on_vehicle = sum(box[_p, w, s, v] for _p in range(number_products))
            model.Add(boxes_shipped_on_vehicle <= capacities[v]).OnlyEnforceIf(trip[w, s, v])
            model.Add(boxes_shipped_on_vehicle == 0).OnlyEnforceIf(trip[w, s, v].Not())

# b) Limit the number of trips each van can make and check that a van was used
for v in range(number_vehicles):
    trips_per_vehicle = sum(trip[_w, _s, v] for (_w, _s) in product(range(number_warehouses), range(number_stores)))
    model.Add(trips_per_vehicle <= trip_limit).OnlyEnforceIf(is_vehicle_used[v])
    model.Add(trips_per_vehicle == 0).OnlyEnforceIf(is_vehicle_used[v].Not())

    # The number of employees assigned to a van needs to 2, 1 or 0
    employees_in_vehicle = sum(is_employee_assigned[v, _e] for _e in range(number_employees))
    model.Add(employees_in_vehicle <= 2).OnlyEnforceIf(is_vehicle_used[v])
    model.Add(employees_in_vehicle == 0).OnlyEnforceIf(is_vehicle_used[v].Not())

    #Check which pair of employees are conflicted in order to pay the penalty
    #for _, (e1, e2) in enumerate(conflicted_employees):
    #    partners = [is_employee_assigned[v, e1], is_employee_assigned[v, e2]]
    #    model.AddBoolAnd(partners).OnlyEnforceIf(is_vehicle_conflicted[v, _])

# An employee assigned to a van can't be assigned to another van
for e in range(number_employees):
    vehicle_options_for_employee = sum(is_employee_assigned[_v, e] for _v in range(number_vehicles))
    model.Add(vehicle_options_for_employee <= 1)

In [180]:
# Define the objective function previously mentioned:
objective_function = []

# First term -> sum of variable costs of transportation
for p in range(number_products):
    for w in range(number_warehouses):
        for s in range(number_stores):
            for v in range(number_vehicles):
                objective_function.append(costs_shipping[w, s] * box[p, w, s, v])

# Second term -> sum of fixed costs of using a specific van
for v in range(number_vehicles):
    objective_function.append(van_fees[v] * is_vehicle_used[v])

# Third term -> sum of salary payments per employee assigned to a task
for v in range(number_vehicles):
    for e in range(number_employees):
        objective_function.append(fixed_salary * is_employee_assigned[v, e])

# Fourth term -> sum of penalties for not avoiding the joint assignment of conflicting employees
for v in range(number_vehicles):
    for _ in range(len(conflicted_employees)):
        objective_function.append(conflict_penalty * is_vehicle_conflicted[v, _])

# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 6
solver.parameters.max_time_in_seconds = 2*60
model.Minimize(sum(objective_function))

### Results

In [182]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status == "OPTIMAL":
    print(f'Solution: Total cost = ${solver.ObjectiveValue()} dollars')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.07 s, objective = 1459
Solution 1, time = 0.07 s, objective = 1429
Solution 2, time = 0.12 s, objective = 1427
Solution 3, time = 0.13 s, objective = 1419
Solution 4, time = 5.61 s, objective = 1289
Solution 5, time = 6.13 s, objective = 1283
Solution 6, time = 6.72 s, objective = 1282
Solution: Total cost = $1282.0 dollars


#### Results extraction

In [183]:
data = []
for (p, w, s, v), or_var in box.items():
    n = or_var.Name()
    x = solver.Value(or_var)
    data.append(["box", n, x, p, w, s, v])

names = [
    "type",
    "name",
    "value",
    "prod",
    "ware",
    "store",
    "vehi",
]

df = pd.DataFrame(data, columns=names)

In [184]:
df[df["prod"]==0].sum(numeric_only=True)

value    244
prod       0
ware     720
store    240
vehi     240
dtype: int64

In [185]:
for (p, s) in product(range(number_products), range(number_stores)):
    total = df[(df['value'] > 0) & (df["store"] == s) & (df["prod"] == p)]
    assert demands[p][s] == total.sum(numeric_only=True).value, "Error"


In [186]:
df[(df['value'] > 0) & (df["store"] == 0) & (df["prod"] == 0)]

,type,name,value,prod,ware,store,vehi
1,box,x_0_0_0_1,20,0,0,0,1
3,box,x_0_0_0_3,8,0,0,0,3
82,box,x_0_5_0_2,7,0,5,0,2
114,box,x_0_7_0_2,6,0,7,0,2
115,box,x_0_7_0_3,15,0,7,0,3


In [7]:
# Trucks to be used
for 
list(results[(results['Variable']=='T')&(results['Value']==1)].Name)

['T_1', 'T_2', 'T_3']

In [8]:
# Trips of van 1

trips_df=results[(results['Variable']=='Z')&(results['Value']>0)]

trips_van_1=[]
for w in range(number_warehouses):
    for s in range(number_stores):
        for v in range(number_vehicles):
            if v==0:
                trips_van_1.append(str(Z[w,s,v]))


display(trips_df[trips_df['Name'].isin(trips_van_1)])

,Variable,Indices,Name,Value
180,Z,Z_warehouse_store_van,Z_1_1_1,1
182,Z,Z_warehouse_store_van,Z_1_3_1,1
184,Z,Z_warehouse_store_van,Z_2_2_1,1
187,Z,Z_warehouse_store_van,Z_3_2_1,1
188,Z,Z_warehouse_store_van,Z_3_3_1,1


In [9]:
# Boxes transported by van 1 during the trip from warehouse 2 to store 2
transport_df = results[(results['Variable']=='X')&(results['Value']>0)]

transport_trip_2_2 = []

for p in range(number_products):
    for w in range(number_warehouses):
        for s in range(number_stores):
            for v in range(number_vehicles):
                if w==1 and s==1 and v==0:
                    transport_trip_2_2.append(str(x[p,w,s,v]))

display(transport_df[transport_df['Name'].isin(transport_trip_2_2)])

,Variable,Indices,Name,Value
52,X,X_product_warehouse_store_van,x_2_2_2_1,4
88,X,X_product_warehouse_store_van,x_3_2_2_1,1


In [10]:
employees_van_1=[]
for v in range(number_vehicles):
    for e in range(number_employees):
        if v==0:
            employees_van_1.append(str(A[v,e]))
            
employees_df=results[(results['Variable']=='A')&(results['Value']>0)]

display(employees_df[employees_df['Name'].isin(employees_van_1)])

,Variable,Indices,Name,Value
224,A,A_van_employee,A_1_5,1
225,A,A_van_employee,A_1_6,1


In [11]:
# Conflicts
results[(results['Variable']=='G')&(results['Value']>0)]

,Variable,Indices,Name,Value
380,G,G_van_pair,G_2_14,1


In [12]:
# Conflicting employees assigned together to van 2
J_employees_conflicts[13]

(3, 4)

In [13]:
# We should be able to see that A_2_3 and A_2_4 both are equal to 1
results[(results['Variable']=='A')&(results['Value']!=0)]

,Variable,Indices,Name,Value
224,A,A_van_employee,A_1_5,1
225,A,A_van_employee,A_1_6,1
230,A,A_van_employee,A_2_3,1
231,A,A_van_employee,A_2_4,1
242,A,A_van_employee,A_3_7,1
243,A,A_van_employee,A_3_8,1
